코드 병합

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import datetime 

from scipy.stats import chi2_contingency
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score,confusion_matrix
from sklearn.tree import export_graphviz
import graphviz
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from scipy.stats import shapiro
import scipy.stats as stats
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier

from statsmodels.api import Logit
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from scipy.stats import chi2_contingency

import warnings
warnings.filterwarnings("ignore")

path = "C:/Users/cokr3/OneDrive/바탕 화면/이차전지 프로젝트/"

In [ ]:


def eval_class_model(y_test, y_pred):
    
    confusion = confusion_matrix(y_test,y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test,y_pred)
    recall = f1_score(y_test, y_pred)
    f1 = f1_score(y_test,y_pred)
    AUC = roc_auc_score(y_test,y_pred)
    print('혼돈행렬 : \n{}'.format(confusion))
    print('accuracy : {:.4f}'.format(accuracy))
    print('precision : {:.4f}'.format(precision))
    print('recall : {:.4f}'.format(recall))
    print('f1 : {:.4f}'.format(f1))
    print('AUC : {:.4f}'.format(AUC))
def chi(df, y,*col):
    crosstab_result = pd.crosstab( index=  df[y], columns = [ df[i] for i in col ])
    chi2_stat, p_val, dof, expected = chi2_contingency(crosstab_result)

    print("Chi-square statistic:", chi2_stat)
    print("P-value:", p_val)
    print("Degrees of freedom:", dof)
    print()
def dt(x, step, depth) :
    
    df_train,df_test = train_test_split(x, test_size = 0.3, stratify = x['y'],random_state = 1234)
    df_train_x = df_train.drop('y',axis = 1)
    df_train_y = df_train['y']

    df_test_x = df_test.drop('y',axis = 1)
    df_test_y = df_test['y']
    print(df_train_x.shape,df_test_x.shape)

    tree_uncust = DecisionTreeClassifier()
    tree_uncust.fit(df_train_x,df_train_y)

    print('Score on training set : {:.3f}'.format(tree_uncust.score(df_train_x,df_train_y)))
    print('Score on test set : {:.3f}'.format(tree_uncust.score(df_test_x,df_test_y)))
    
    v_feature_name = df_train_x.columns
    tree_final = DecisionTreeClassifier(max_depth = depth)
    tree_final.fit(df_train_x,df_train_y)

    train_score = []
    test_score = [] 

    train_score.append(tree_final.score(df_train_x, df_train_y))
    print('train data Accuray', train_score)

    test_score.append(tree_final.score(df_test_x, df_test_y))
    print('test data Accuray', test_score)

    pred = tree_final.predict(df_test_x)

    tree_final.score(df_test_x, df_test_y) 
    accuracy_score(df_test_y, pred) 

    train_pred = tree_final.predict(df_train_x)
    test_pred = tree_final.predict(df_test_x)

    print('Train 성능')
    eval_class_model(df_train_y,train_pred)

    print('Test 성능')
    eval_class_model(df_test_y, test_pred)

    export_graphviz(tree_final, out_file = 'posco_tree_final2.png',
                   feature_names = v_feature_name, impurity = True, filled = True)
    try :
        with open('posco_tree_final2.png') as f:
            dot_graph = f.read()
            
        graph = graphviz.Source(dot_graph)
        graph.render(filename = f"{path}{step}", format = "png")
    except :
        pass

In [ ]:
#%% 00. RAW data
df_raw_tat = pd.read_csv(f"{path}bat_tat.csv")
df_raw_process = pd.read_csv(f"{path}bat_process.csv", encoding = 'cp949')

In [ ]:
#%% 01. 데이터 전처리
# 중복제거 : X
# 결측치 제거(df_process)
df_process_na = df_raw_process[df_raw_process.isna().any(axis=1)] 
df_process = df_raw_process.drop(df_process_na[ df_process_na["judge"] == "양품"].index, axis = 0).reset_index(drop = True)
df_process.rename({'judge' : "y"}, inplace = True, axis = 1)
df_process.replace({"양품" : 1, "불량" : 0}, inplace = True)

In [ ]:
# 이상치 제거(df_tat)
df_tat = df_raw_tat.copy()

#1. rta2_tat
print("rta2_tat 이상치 제거 전 : ", df_tat.shape[0])
df_tat= df_tat[ df_tat["rta2_tat"] >= 6000 ].reset_index(drop = True)
print("rta2_tat 이상치 제거 후 : ", df_tat.shape[0], '\n')

#2. ocv1_tat
print("ocv1_tat 이상치 제거 전 : ", df_tat.shape[0])
df_tat = df_tat[ df_tat["ocv1_tat"] <= 140 ].reset_index(drop = True)
print("ocv1_tat 이상치 제거 후 : ", df_tat.shape[0], '\n')

#3. c1_tat
print("c1_tat 이상치 제거 전 : ", df_tat.shape[0])
df_tat = df_tat[ df_tat["c1_tat"] <= 6000 ].reset_index(drop = True)
print("c1_tat 이상치 제거 후 : ", df_tat.shape[0], '\n')

#4. c2_tat
print("c2_tat 이상치 제거 전 : ", df_tat.shape[0])
df_tat = df_tat[ df_tat["c2_tat"] <= 1000 ].reset_index(drop = True)
print("c2_tat 이상치 제거 후 : ", df_tat.shape[0], '\n')

#5. dc2_tat
print("dc2_tat 이상치 제거 전 : ", df_tat.shape[0])
df_tat = df_tat[ df_tat["dc2_tat"] <= 460 ].reset_index(drop = True)
print("dc2_tat 이상치 제거 후 : ", df_tat.shape[0], '\n')

#6. c3_tat
print("c3_tat 이상치 제거 전 : ", df_tat.shape[0])
df_tat = df_tat[ df_tat["c3_tat"] <= 10000 ].reset_index(drop = True)
print("c3_tat 이상치 제거 후 : ", df_tat.shape[0], '\n')

#7. dc3_tat
print("dc3_tat 이상치 제거 전 : ", df_tat.shape[0])
df_tat = df_tat[ df_tat["dc3_tat"] <= 7000 ].reset_index(drop = True)
print("dc3_tat 이상치 제거 후 : ", df_tat.shape[0], '\n')

#8. c4_tat
print("c4_tat 이상치 제거 전 : ", df_tat.shape[0])
df_tat = df_tat[ df_tat["c4_tat"] <= 14000 ].reset_index(drop = True)
print("c4_tat 이상치 제거 후 : ", df_tat.shape[0], '\n')

#9. pg1_tat
print("pg1_tat 이상치 제거 전 : ", df_tat.shape[0])
df_tat = df_tat[ df_tat["pg1_tat"] <= 800 ].reset_index(drop = True)
df_tat = df_tat[ df_tat["pg1_tat"] >= 200 ].reset_index(drop = True)
print("pg1_tat 이상치 제거 후 : ", df_tat.shape[0], '\n')

#10. sa1_tat
print("sa1_tat 이상치 제거 전 : ", df_tat.shape[0])
df_tat = df_tat[ df_tat["sa1_tat"] >= 2000 ].reset_index(drop = True)
print("sa1_tat 이상치 제거 후 : ", df_tat.shape[0], '\n')

#11. sa2_tat
print("sa2_tat 이상치 제거 전 : ", df_tat.shape[0])
df_tat = df_tat[ df_tat["sa2_tat"] >= 250000 ].reset_index(drop = True)
print("sa2_tat 이상치 제거 후 : ", df_tat.shape[0], '\n')

#12. sa3_tat
print("sa3_tat 이상치 제거 전 : ", df_tat.shape[0])
df_tat = df_tat[ df_tat["sa3_tat"] >= 190000 ].reset_index(drop = True)
print("sa3_tat 이상치 제거 후 : ", df_tat.shape[0], '\n')

del df_process_na

In [ ]:
#%% 01-2. error_rate
tray_list = pd.merge( df_raw_process[["lot_id",'tray_id']], pd.DataFrame( df_raw_process.groupby(["lot_id","tray_id"])['cell_id'].count()).reset_index(drop = False) )
tray_list = pd.merge( tray_list, df_raw_process[ df_raw_process["judge"] == "불량" ].groupby(["lot_id","tray_id"])['cell_id'].count().reset_index(drop = False) ,
                     on = ["lot_id","tray_id"], how = "outer")
tray_list.fillna(0, inplace = True)
tray_list["error_rate"] = tray_list[tray_list.columns[3]] / tray_list[tray_list.columns[2]] # 브로드 캐스팅
tray_list.drop_duplicates(["lot_id", "tray_id", "error_rate"], keep = "first", inplace = True)

tray_list_cut = tray_list[ tray_list["cell_id_x"] >= 20 ]

df_tat = pd.merge( df_tat, tray_list_cut[["lot_id", "tray_id", "error_rate"]],
                  on = ["lot_id", "tray_id"],
                  how = 'inner')
df_tat = df_tat[ df_raw_tat.columns.insert(2, "error_rate").to_list() ] # y = 불량률(파생변수)

df_process_rate = pd.merge( df_process, tray_list_cut[["lot_id", "tray_id", "error_rate"]],
                             on = ["lot_id", "tray_id"],
                             how = "outer").drop('y', axis = 1)
df_process_rate = df_process_rate[ df_process_rate.columns.insert(3, "error_rate").to_list() ] # y = 불량률(파생변수)

In [ ]:
#%% 02. 데이터 분석
#%% 02-1. OCV 유의성 검사
"""
ocv와 chi_분석(small-y)
ocv 기준 구하기 -> "조기 경보로 이용"
"""

print("ocv1과 불량 판정에 대한 유의성 검정")
chi(df_process, "y", "ocv1_ocv")

print("ocv2과 불량 판정에 대한 유의성 검정")
chi(df_process, "y", "ocv2_ocv")

#%% 02-1. OCV 특성 검사
import statsmodels.formula.api as smf

df = []

for k in range(1,5) :

    df_ocv = df_process[["ocv1_ocv"] + [i for i in df_process.columns if f"c{k}" in i and "pc" not in i and f"dc{k}" not in i]].select_dtypes(exclude = "object")
    
    scaler = StandardScaler()
    
    df_x_scaled = scaler.fit_transform(df_ocv.drop("ocv1_ocv", axis = 1 ))
    df_x_scaled = pd.DataFrame(df_x_scaled, columns= df_ocv.drop("ocv1_ocv", axis = 1 ).columns )
    
    df_x_scaled = pd.concat([ df_ocv["ocv1_ocv"], df_x_scaled ], axis = 1)
    
    df.append(df_x_scaled)
    
    if k == 4 : break
    
    df_ocv = df_process[["ocv1_ocv"] + [i for i in df_process.columns if f"dc{k}" in i ]].select_dtypes(exclude = "object")

    df_x_scaled = scaler.fit_transform(df_ocv.drop("ocv1_ocv", axis = 1 ))
    df_x_scaled = pd.DataFrame(df_x_scaled, columns= df_ocv.drop("ocv1_ocv", axis = 1 ).columns )
    
    df_x_scaled = pd.concat([ df_ocv["ocv1_ocv"], df_x_scaled ], axis = 1)
    
    df.append(df_x_scaled)
    

for i,j in zip( df, ["C1", "dc1", "c2", "dc2", "c3", "dc3", "c4"] ):
    print(j)

    df_train, df_test = train_test_split(i, test_size = 0.3)
    
    print("train data size : {}".format(df_train.shape))
    print("test data size : {}".format(df_test.shape))
    
    
    fx = "ocv1_ocv ~ +" + "+".join(i.columns[1:])
    reg_model = smf.ols(formula = fx, data = i)
    
    reg_result = reg_model.fit()
    print(reg_result.summary())

In [ ]:
#%% 02-2. 설비유의차 good/bad1/bad2

df_rcd = pd.concat( [ df_process[["lot_id", "tray_id", "y"]],
                     df_process[ [ i for i in df_process.columns if "c1_box" in i and "dc1" not in i and "pc1" not in i]],
                     df_process[ [ i for i in df_process.columns if "c1_temp" in i and "dc1" not in i and "pc1" not in i]]],
                   axis = 1)

df_bad1 = df_rcd.loc[ df_rcd["c1_box_col"].isin(["20열", "21열", "22열"])]
df_bad1 = df_bad1.loc[ df_bad1["c1_box_dan"].isin(["01단", '02단', "03단", "04단"])].reset_index(drop = True)

df_bad2 = df_rcd.loc[ df_rcd["c1_box_col"].isin(["11열", "12열", "13열", "14열"])]
df_bad2 = df_bad2.loc[ df_bad2["c1_box_dan"].isin(["09단", '10단', "11단"])].reset_index(drop = True)

df_good = df_rcd.loc[ df_rcd["c1_box_col"].isin(["11열", "12열", "13열", "14열"])]
df_good = df_good.loc[ df_good["c1_box_dan"].isin(["05단", '06단', "07단"])].reset_index(drop = True)



plt.figure(figsize=(10, 6))

# 각 데이터프레임에 대한 히스토그램 그리기
sns.histplot(data=df_bad1, x='c1_temp_avg', hue='y', multiple='stack', bins=5, alpha=0.5, palette="Paired")
sns.histplot(data=df_bad2, x='c1_temp_avg', hue='y', multiple='stack', bins=5, alpha=0.5, palette="Set2")
sns.histplot(data=df_good, x='c1_temp_avg', hue='y', multiple='stack', bins=5, alpha=0.5)

plt.title('Temperature vs Defects Histogram')
plt.xlabel('Temperature')
plt.ylabel('Frequency')
plt.legend(title='Data')
plt.show()
#%% 02-3. C1 설명변수별 chi통계량
# 충전 1단계에서 temp_avg, time_cc, capa, voltage_avg

chi_c1 = df_process[ ["y",'c1_temp_avg', "c1_time_cc", "c1_capa", "c1_voltage_avg"]]
    
for i in ['c1_temp_avg', "c1_time_cc", "c1_capa", "c1_voltage_avg"] : 
        
    crosstab_result = pd.crosstab(index=  chi_c1['y'], columns= chi_c1[i])
    chi2_stat, p_val, dof, expected = chi2_contingency(crosstab_result)

    print("Chi-square statistic:", chi2_stat)
    print("P-value:", p_val)
    print("Degrees of freedom:", dof)
    print()

In [ ]:
#%% temp_time
df_temp_time = pd.read_excel(f"{path}temp_tat.xlsx")
df_temp_time_c1 = df_temp_time[["error_rate"] + [i for i in df_temp_time.columns if "c1" in i and "dc" not in i and "pc" not in i  ]]

In [ ]:
#%% 02-4. DT

df_process_tat = pd.merge(df_process, df_tat.drop("error_rate", axis = 1), on = ['lot_id', "tray_id"], how = "inner")

col = [ i for i in df_process_tat.columns if "temp" in i][:-1]
co2 = [ i for i in df_process_tat.columns if "_tat" in i and "c" in i and "pc" not in i and "ocv" not in i]

df_A = []

for x, y in zip(col, co2) :
    df_A.append(pd.concat(  [df_process_tat[["y"]], df_process_tat[[x,y]]] , axis = 1))
    
for x, j in zip(df_A, ["c1", "dc1", "c2", "dc2", "c3", "dc3", "c4"])    :
    print(j)
    dt(x, j, 3)

In [ ]:
#%% 온도별 histogram

data = df_process[["y"]]

# 각 데이터프레임에 대한 히스토그램 그리기
sns.histplot(data=df_bad1, x='c1_temp_avg', hue='y', multiple='stack', bins=5, alpha=0.5, palette="Paired")
sns.histplot(data=df_bad2, x='c1_temp_avg', hue='y', multiple='stack', bins=5, alpha=0.5, palette="Set2")
sns.histplot(data=df_good, x='c1_temp_avg', hue='y', multiple='stack', bins=5, alpha=0.5)

plt.title('Temperature vs Defects Histogram')
plt.xlabel('Temperature')
plt.ylabel('Frequency')
plt.legend(title='Data')
plt.show()

#%%

In [ ]:
"""
귀무가설 설비유의차 row,col은 양품/불량간 관계가 없다를 충분히 기각하므로,

설비간 유의차가 있다.
"""

df_chi_all = df_process[["y", "c1_box_col", "c1_box_dan"]]

crosstab_result = pd.crosstab(index=df_chi_all ['y'], columns=[df_chi_all['c1_box_col'], df_chi_all['c1_box_dan']])
chi2_stat, p_val, dof, expected = chi2_contingency(crosstab_result)

# 결과 출력
print("Chi-square statistic:", chi2_stat)
print("P-value:", p_val)
print("Degrees of freedom:", dof)

In [ ]:
#%% process별 불량률

tray_list = pd.merge( df_raw_process[["lot_id",'tray_id']], pd.DataFrame( df_raw_process.groupby(["lot_id","tray_id"])['cell_id'].count()).reset_index(drop = False) )
tray_list = pd.merge( tray_list, df_raw_process[ df_raw_process["judge"] == "불량" ].groupby(["lot_id","tray_id"])['cell_id'].count().reset_index(drop = False) ,
                     on = ["lot_id","tray_id"], how = "outer")
tray_list.fillna(0, inplace = True)
tray_list["error_rate"] = tray_list[tray_list.columns[3]] / tray_list[tray_list.columns[2]] # 브로드 캐스팅
tray_list.drop_duplicates(["lot_id", "tray_id"], keep = "first", inplace = True)
tray_list_cut = tray_list[ tray_list["cell_id_x"] >= 20 ]
#%%

df_process_  = pd.merge(df_process, tray_list_cut[["lot_id", "tray_id", "error_rate"]],
                        on = ["lot_id", "tray_id"],
                        how = "inner")

In [ ]:
#%% C구역

df_rcd = df_process.select_dtypes(include = "object")
df_rcd.drop_duplicates(['lot_id', "tray_id"], keep="first", inplace = True)

df_rcd = pd.merge(tray_list_cut[["lot_id", "tray_id", "error_rate"]], df_rcd,
                  on = ["lot_id", "tray_id"],
                  how = "inner")
#%%
df_rcd_c = df_rcd[["lot_id", "tray_id", "c1_box_col", "c1_box_dan"]]

df_rcd_c = pd.merge( tray_list_cut , df_rcd_c, on = ["lot_id", "tray_id"], how = "inner")

df_rcd_bad = df_rcd_c.loc[ df_rcd_c["c1_box_dan"].isin(["01단", "02단", "03단", "04단"])]
df_rcd_bad = df_rcd_bad.loc[ df_rcd_bad["c1_box_col"].isin(["22열", "21열", "20열"])]

print(df_rcd_bad["error_rate"].mean())
print(df_rcd_bad['cell_id_y'].sum())

#%% A열

df_rcd_good = df_rcd_c.loc[ df_rcd_c["c1_box_dan"].isin(["08단", "09단", "10단", "11단"])]
df_rcd_good = df_rcd_good.loc[ df_rcd_good["c1_box_col"].isin(["14열", "13열", "12열"])]

print(df_rcd_good["error_rate"].mean())

#%% B 구역

df_rcd_b = df_rcd_c.loc[ df_rcd_c["c1_box_dan"].isin(["04단", "05단", "06단", "07단"])]
df_rcd_b = df_rcd_b.loc[ df_rcd_b["c1_box_col"].isin(["10열", "12열", "11열"])]

print(df_rcd_b["error_rate"].mean())

#%% bad 구역(c구역 DT) - process

df_process_bad = df_process.loc[ df_process["c1_box_dan"].isin(["01단", "02단", "03단", "04단"])]
df_process_bad = df_process_bad.loc[ df_process_bad["c1_box_col"].isin(["22열", "21열", "20열"])]
df_process_bad = pd.concat( [ df_process_bad[["y"]], df_process_bad[ [i for i in df_process_bad.columns if ( "c1_" in i 
                                 or "c2_" in i or "c3_" in i or "c4_" in i )][:-5] ].select_dtypes(exclude = "object")], axis = 1) 

df_process_bad["y"].replace({"양품" : 1, "불량" : 0}, inplace = True)

In [ ]:
#%% ----------------------------------
#%% DT_c1

df_process_bad_c1 = df_process_bad[ df_process_bad.columns[:7]]

In [ ]:
#%% DT_datasplit
df_train,df_test = train_test_split(df_process_bad_c1, test_size = 0.3, stratify = df_process_bad_c1['y'],random_state = 1234)
df_train_x = df_train.drop('y',axis = 1)
df_train_y = df_train['y']

df_test_x = df_test.drop('y',axis = 1)
df_test_y = df_test['y']
print(df_train_x.shape,df_test_x.shape)

In [ ]:
#%% default

tree_uncust = DecisionTreeClassifier()
tree_uncust.fit(df_train_x,df_train_y)

print('Score on training set : {:.3f}'.format(tree_uncust.score(df_train_x,df_train_y)))
print('Score on test set : {:.3f}'.format(tree_uncust.score(df_test_x,df_test_y)))

In [ ]:
#%% max_depth

# train 및 test 정확도 결과 저장용
train_accuracy = []; test_accuracy = []
# min_samples_leaf: 잎사귀 수 제한
para_depth = [d for d in range(1, 8)]

for d in para_depth:
    dt = DecisionTreeClassifier(max_depth = d)
    dt.fit(df_train_x, df_train_y)
    train_accuracy.append(dt.score(df_train_x, df_train_y))
    test_accuracy.append(dt.score(df_test_x, df_test_y))

# 데이터 테이블로 저장
df_accuracy_depth = pd.DataFrame()
df_accuracy_depth["MaxDepth"] = para_depth
df_accuracy_depth["TrainAccuracy"] = train_accuracy
df_accuracy_depth["TestAccuracy"] = test_accuracy

# 모델 정확도 그래프 확인
print(df_accuracy_depth)
plt.plot(para_depth, train_accuracy, linestyle = "-", label = "Train Accuracy")
plt.plot(para_depth, test_accuracy, linestyle = "--", label = "Test Accuracy")
plt.xlabel("max_depth"); plt.ylabel("accuracy")
plt.legend()

In [ ]:
# max_depth = 4

#%% min_split

train_accuracy = []; test_accuracy = []

# min_samples_leaf: 잎사귀 수 제한
para_split = [s*2 for s in range(10, 20)]

for split_ in para_split:
    dt = DecisionTreeClassifier(max_depth = 4, min_samples_split = split_)
    dt.fit(df_train_x, df_train_y)
    train_accuracy.append(dt.score(df_train_x, df_train_y))
    test_accuracy.append(dt.score(df_test_x, df_test_y))

# 데이터 테이블로 저장
df_accuracy_leaf = pd.DataFrame()
df_accuracy_leaf["MinSamplesSplit"] = para_split
df_accuracy_leaf["TrainAccuracy"] = train_accuracy
df_accuracy_leaf["TestAccuracy"] = test_accuracy

# 모델 정확도 그래프 확인
print(df_accuracy_leaf)
plt.plot(para_split, train_accuracy, linestyle = "-", label = "Train Accuracy")
plt.plot(para_split, test_accuracy, linestyle = "--", label = "Test Accuracy")
plt.xlabel("min_samples_split"); plt.ylabel("accuracy")
plt.legend()

In [ ]:
# default

#%% min_leaf

train_accuracy = []; test_accuracy = []

# min_samples_leaf: 잎사귀 수 제한
para_leaf = [n_leaf for n_leaf in range(1, 41)]

for v_min_samples_leaf in para_leaf:
    dt = DecisionTreeClassifier(max_depth = 4, min_samples_leaf = v_min_samples_leaf)
    dt.fit(df_train_x, df_train_y)
    train_accuracy.append(dt.score(df_train_x, df_train_y))
    test_accuracy.append(dt.score(df_test_x, df_test_y))

# 데이터 테이블로 저장
df_accuracy_leaf = pd.DataFrame()
df_accuracy_leaf["MinSamplesLeaf"] = para_leaf
df_accuracy_leaf["TrainAccuracy"] = train_accuracy
df_accuracy_leaf["TestAccuracy"] = test_accuracy

# 모델 정확도 그래프 확인
print(df_accuracy_leaf)
plt.plot(para_leaf, train_accuracy, linestyle = "-", label = "Train Accuracy")
plt.plot(para_leaf, test_accuracy, linestyle = "--", label = "Test Accuracy")
plt.xlabel("min_samples_leaf"); plt.ylabel("accuracy")
plt.legend()

In [ ]:
#%% 최종^^

v_feature_name = df_train_x.columns
tree_final = DecisionTreeClassifier(max_depth = 4)
tree_final.fit(df_train_x,df_train_y)

train_score = []
test_score = [] 

# 학습 데이터로 모델 정확도 측정 
train_score.append(tree_final.score(df_train_x, df_train_y))
print('train data Accuray', train_score)

test_score.append(tree_final.score(df_test_x, df_test_y))
print('test data Accuray', test_score)

pred = tree_final.predict(df_test_x)
# 테스트 데이터로 모델 정확도 측정 
tree_final.score(df_test_x, df_test_y) 
accuracy_score(df_test_y, pred) 

train_pred = tree_final.predict(df_train_x)
test_pred = tree_final.predict(df_test_x)

print('Train 성능')
eval_class_model(df_train_y,train_pred)

print('Test 성능')
eval_class_model(df_test_y, test_pred)

In [ ]:
#%% IV

v_feature_name = df_train_x.columns

df_importance = pd.DataFrame()
df_importance['Feature'] = v_feature_name
df_importance['Importance'] = tree_final.feature_importances_

df_importance.sort_values('Importance', ascending = False, inplace = True)
df_importance.round(3)

In [ ]:
# 설명변수 중요도 그래프
df_importance = df_importance[df_importance["Importance"] > 0.005]
df_importance.sort_values('Importance', ascending = True, inplace = True)
coordinates = range(len(df_importance))
plt.barh(y = coordinates, width = df_importance['Importance'])
plt.yticks(coordinates, df_importance['Feature'])
plt.xlabel('feature_importance')
plt.ylabel('feature')
#%% plot

export_graphviz(tree_final, out_file = 'posco_tree_final2.png',
               feature_names = v_feature_name, impurity = True, filled = True)

with open('posco_tree_final2.png') as f:
    dot_graph = f.read()
    
print(graphviz.Source(dot_graph))

In [ ]:
#%% --------------------------------------
#%% DataSet

df_process_A = df_process.drop(["lot_id", "tray_id", "cell_id", "dt_start","rta1_cell_no"] + [i for i in df_process.columns if "box" in i or "dan" in i or "row" in i], axis = 1)
df_process_A.replace({'양품' : 1, "불량" : 0}, inplace = True)

df_process_first = df_process_A[ df_process_A.columns[:2]  ]
df_process_second = pd.concat( [ df_process_A[["judge"]] ,df_process_A[ df_process_A.columns[2:46] ] ], axis = 1)
df_process_third = pd.concat([ df_process_A[["judge"]] ,df_process_A[ df_process_A.columns[46:55] ] ], axis = 1)

In [ ]:
#%% tray_온도 ----------------------------------------------------------------------------

df_temp = df_process[["lot_id", "tray_id"] + [i for i in df_process.columns if "temp" in i]]
df_temp.drop_duplicates(df_temp.columns, keep = "first" , inplace = True)

df_temp_time = pd.merge(df_tat, df_temp, on = ["lot_id", "tray_id"], how = "inner")

df_dt_all = df_process[ ["y"]+ [ i for i in df_process.columns if "temp" in i ][:-1]]
df_dt_all.replace({'양품' : 1, "불량" : 0}, inplace = True)

In [ ]:
#%% DT - dataset
df_train,df_test = train_test_split(df_dt_all, test_size = 0.3, stratify = df_dt_all['y'],random_state = 1234)
df_train_x = df_train.drop('y',axis = 1)
df_train_y = df_train['y']

df_test_x = df_test.drop('y',axis = 1)
df_test_y = df_test['y']
print(df_train_x.shape,df_test_x.shape)

In [ ]:
#%%
v_feature_name = df_train_x.columns
tree_final = DecisionTreeClassifier(max_depth = 4)
tree_final.fit(df_train_x,df_train_y)

train_score = []
test_score = [] 

# 학습 데이터로 모델 정확도 측정 
train_score.append(tree_final.score(df_train_x, df_train_y))
print('train data Accuray', train_score)

test_score.append(tree_final.score(df_test_x, df_test_y))
print('test data Accuray', test_score)

pred = tree_final.predict(df_test_x)

In [ ]:
# 테스트 데이터로 모델 정확도 측정 
tree_final.score(df_test_x, df_test_y) 
accuracy_score(df_test_y, pred) 

train_pred = tree_final.predict(df_train_x)
test_pred = tree_final.predict(df_test_x)

print('Train 성능')
eval_class_model(df_train_y,train_pred)

print('Test 성능')
eval_class_model(df_test_y, test_pred)

In [ ]:
#%% IV

v_feature_name = df_train_x.columns

df_importance = pd.DataFrame()
df_importance['Feature'] = v_feature_name
df_importance['Importance'] = tree_final.feature_importances_

df_importance.sort_values('Importance', ascending = False, inplace = True)
df_importance.round(3)

In [ ]:
# 설명변수 중요도 그래프
df_importance = df_importance[df_importance["Importance"] > 0.005]
df_importance.sort_values('Importance', ascending = True, inplace = True)
coordinates = range(len(df_importance))
plt.barh(y = coordinates, width = df_importance['Importance'])
plt.yticks(coordinates, df_importance['Feature'])
plt.xlabel('feature_importance')
plt.ylabel('feature')

In [ ]:
#%%
x = tray_list.copy()
x = x[ x['cell_id_y'] == 1 ]
x.drop_duplicates(["cell_id_x"], keep = "first", inplace = True)
x.reset_index(drop = True, inplace = True)
x.rename({"cell_id_x" : "tray별 cell 개수" , "cell_id_y" : "tray별 불량 개수"}, axis = 1, inplace = True)
x.sort_values(['tray별 cell 개수'], ascending = False, inplace = True)
x["error_rate"] = x["error_rate"].round(3)

x["tray_id"] = "tray"

In [ ]:
def dt(x, step) :
    
    df_train,df_test = train_test_split(x, test_size = 0.3, stratify = x['y'],random_state = 1234)
    df_train_x = df_train.drop('y',axis = 1)
    df_train_y = df_train['y']

    df_test_x = df_test.drop('y',axis = 1)
    df_test_y = df_test['y']
    print(df_train_x.shape,df_test_x.shape)

    tree_uncust = DecisionTreeClassifier()
    tree_uncust.fit(df_train_x,df_train_y)

    print('Score on training set : {:.3f}'.format(tree_uncust.score(df_train_x,df_train_y)))
    print('Score on test set : {:.3f}'.format(tree_uncust.score(df_test_x,df_test_y)))
    
    v_feature_name = df_train_x.columns
    tree_final = DecisionTreeClassifier(max_depth = 4)
    tree_final.fit(df_train_x,df_train_y)

    train_score = []
    test_score = [] 

    # 학습 데이터로 모델 정확도 측정 
    train_score.append(tree_final.score(df_train_x, df_train_y))
    print('train data Accuray', train_score)

    test_score.append(tree_final.score(df_test_x, df_test_y))
    print('test data Accuray', test_score)

    pred = tree_final.predict(df_test_x)
    # 테스트 데이터로 모델 정확도 측정 
    tree_final.score(df_test_x, df_test_y) 
    accuracy_score(df_test_y, pred) 

    train_pred = tree_final.predict(df_train_x)
    test_pred = tree_final.predict(df_test_x)

    print('Train 성능')
    eval_class_model(df_train_y,train_pred)

    print('Test 성능')
    eval_class_model(df_test_y, test_pred)

    export_graphviz(tree_final, out_file = 'posco_tree_final2.png',
                   feature_names = v_feature_name, impurity = True, filled = True)

    with open('posco_tree_final2.png') as f:
        dot_graph = f.read()
        
    graph = graphviz.Source(dot_graph)
    graph.render(filename = f"{path}{step}", format = "png")
 

In [ ]:
#%% dt 한 번에 돌리기 스킬~!
df_process_A_cd = df_process_A.copy()
df_process_A_cd.drop([i for i in df_process_A_cd.columns if "pc" in i], inplace = True, axis = 1)

df_A = []

for j in range(1,5) :
    x = df_process_A_cd[ ["y"] + [i for i in df_process_A_cd.columns if f"c{j}_temp" in i and f"dc{j}_temp" not in i]]
    df_A.append(x)
    
    if j == 4 :
        break
    x = df_process_A_cd[ ["y"] + [i for i in df_process_A_cd.columns if f"dc{j}_temp" in i]]
    df_A.append(x)
      
for x, j in zip(df_A, ["c1", "dc1", "c2", "dc2", "c3", "dc3", "c4"])    :
    dt(x, "c1")

In [ ]:
#%% F-검정

x_df = df_rcd_c.drop(["lot_id", "tray_id", "cell_id_x", "cell_id_y"], axis = 1)

df = []

for i in x_df["c1_box_dan"].unique() : # 열별 유의차
    df.append( x_df[ x_df["c1_box_dan"] == i][["error_rate"]])
data = [x['error_rate'] for x in df]

f, p = stats.f_oneway(*data)
print("F-statistic:", f)
print("p-value:", p)

In [ ]:
#%% 설비유의차를 보고 싶음 --------------------------------
#%% 

chi_c1 = df_process_[ ["y",'c1_temp_avg']]
chi_dc2 = df_process_[ ["y",'dc2_temp_avg']]
chi_c4 = df_process_[ ["y",'c4_temp_avg']]

f_c1= df_temp_time[["error_rate", "c1_temp_avg"]]
f_dc1= df_temp_time[["error_rate", "dc1_temp_avg"]]
f_c2= df_temp_time[["error_rate", "c2_temp_avg"]]

In [ ]:
#%%

for i in [f_c1, f_dc1, f_c2] :
    f, p = stats.f_oneway( i["error_rate"], i[i.columns[1]])
    
    print("F-statistic:", f)
    print("p-value:", p)
    print()
    
for i in [chi_c1, chi_dc2, chi_c4] : 
        
    crosstab_result = pd.crosstab(index=  i['y'], columns= i[i.columns[1]])
    chi2_stat, p_val, dof, expected = chi2_contingency(crosstab_result)

    print("Chi-square statistic:", chi2_stat)
    print("P-value:", p_val)
    print("Degrees of freedom:", dof)
    print()
    
   #%%
   
x = pd.concat([df_raw_process[["lot_id","tray_id",'judge']], df_raw_process[ [ i for i in df_raw_process.columns if "_temp_avg" in i][:-1]]], axis =1)


col = [ i for i in x.columns if "temp" in i]

for i in col :
    x = x[ x[i] >= 257 ].reset_index(drop = True)
    x = x[ x[i] <= 294 ].reset_index(drop = True)
    

x_ = x.drop_duplicates(["lot_id", "tray_id"]).reset_index(drop = True)
x_ = pd.merge(x_[["lot_id", "tray_id"]], tray_list, on = ["lot_id", "tray_id"], how = "inner")
print(x_["error_rate"].mean())
x_.rename({"error_rate" : "y"}, axis = 1, inplace = True)

In [ ]:
#%%

df_train,df_test = train_test_split(x_, test_size = 0.3)
df_train,df_test = train_test_split(df_test, test_size = 0.3)
df_train,df_test = train_test_split(df_test, test_size = 0.3)

print(df_test["y"].mean())

In [ ]:
#%%
df_train_x = df_train.drop('error_rate',axis = 1)
df_train_y = df_train['error_rate']

#%%